In [ ]:
import json
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Sequential

In [6]:
with open('../../assets/annotations.json', 'r') as file:
    annotations = json.load(file)

In [7]:
labels = {'standing': 0, 'takedown': 1, 'open_guard': 2, 'half_guard': 3, 'closed_guard': 4, '5050_guard': 5, 'side_control': 6, 'mount': 7, 'back': 8, 'turtle': 9}
num_labels = len(labels)
num_keypoints = 17
num_players = 2

In [ ]:
from random import shuffle

data = []

try:
    for annotation in annotations:
        label = annotation['position']
        if label[-1].isdigit():
            label = label[:-1]
        
        if annotation.get('pose1'):
            keypoints1 = [keypoint[:2] for keypoint in annotation['pose1']]
        else:
            keypoints1 = [[0] * 2] * num_keypoints
            
        if annotation.get('pose2'):
            keypoints2 = [keypoint[:2] for keypoint in annotation['pose2']]
        else:
            keypoints2 = [[0] * 2] * num_keypoints
            
        keypoints = np.array(keypoints1 + keypoints2).astype(np.float32).reshape(num_keypoints * num_players * 2)        
        max_x = max(keypoints)
        normalized_keypoints = keypoints / max_x
        data.append((normalized_keypoints, labels[label]))
        
        # keypoints com poses invertidas
        inverted_keypoints = np.array(keypoints2 + keypoints1).astype(np.float32).reshape(num_keypoints * num_players * 2)        
        max_x = max(keypoints)
        normalized_keypoints = inverted_keypoints / max_x
        data.append((normalized_keypoints, labels[label]))
    
except Exception as e:
    print("Error:", e)
    
keypoints_list = []
labels_list = []

print(data)

shuffle(data)

for keypoints, label in data:
    keypoints_list.append(keypoints)
    labels_list.append(label)
    
labels_list = np.array(labels_list)

file.close()


In [12]:
TRAIN_PERCENTAGE = 0.8

train_keypoints = keypoints_list[:int(len(keypoints_list) * TRAIN_PERCENTAGE)]
train_labels = labels_list[:int(len(labels_list) * TRAIN_PERCENTAGE)]

test_keypoints = keypoints_list[int(len(keypoints_list) * TRAIN_PERCENTAGE):]
test_labels = labels_list[int(len(labels_list) * TRAIN_PERCENTAGE):]

train_keypoints = np.array(train_keypoints)
train_labels = tf.keras.utils.to_categorical(train_labels)

test_keypoints = np.array(test_keypoints)
test_labels = tf.keras.utils.to_categorical(test_labels)

print("Train: ", len(train_keypoints))
print("Test: ", len(test_keypoints))

print("Label example: ", train_labels[65])
print("Keypoint example", train_keypoints[65])

Train:  192446
Test:  48112
Label example:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Keypoint example [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.9152806  0.2448178
 0.9079395  0.22825775 0.9186173  0.2287743  0.8366856  0.2119278
 0.91341656 0.22078535 0.7652717  0.29898068 0.9087139  0.29523975
 0.7666144  0.4353233  0.94042003 0.4562554  0.9063411  0.4489092
 1.         0.46783373 0.7492086  0.5439522  0.8375664  0.5368985
 0.77531666 0.7837812  0.85116094 0.78205204 0.7588074  0.99223167
 0.8253894  0.99223495]


In [13]:
model = Sequential([
    layers.Dense(512, activation='relu', input_shape=(num_keypoints * 2 * num_players,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_labels, activation='softmax')
])

model.summary()

/home/waizbart/Documents/projects/bjj_ia/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1730129740.070102   32723 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-28 12:35:40.364614: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libr

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │        35,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,898 (429.29 KB)

 Trainable params: 109,898 (429.29 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
checkpoint_path = "weights.best.keras"

checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)

history = model.fit(train_keypoints, train_labels,
                    epochs=100,
                    batch_size=16,
                    validation_data=(test_keypoints, test_labels),
                    callbacks=[checkpoint, earlystopping])

Epoch 1/100
11990/12028 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.5567 - loss: 1.2546
Epoch 1: val_accuracy improved from -inf to 0.74480, saving model to weights.best.keras
12028/12028 ━━━━━━━━━━━━━━━━━━━━ 11s 872us/step - accuracy: 0.5570 - loss: 1.2537 - val_accuracy: 0.7448 - val_loss: 0.7221
Epoch 2/100
11985/12028 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.7500 - loss: 0.6944
Epoch 2: val_accuracy improved from 0.74480 to 0.78812, saving model to weights.best.keras
12028/12028 ━━━━━━━━━━━━━━━━━━━━ 10s 860us/step - accuracy: 0.7501 - loss: 0.6943 - val_accuracy: 0.7881 - val_loss: 0.5889
Epoch 3/100
12016/12028 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - accuracy: 0.7938 - loss: 0.5676
Epoch 3: val_accuracy improved from 0.78812 to 0.79340, saving model to weights.best.keras
12028/12028 ━━━━━━━━━━━━━━━━━━━━ 10s 871us/step - accuracy: 0.7938 - loss: 0.5676 - val_accuracy: 0.7934 - val_loss: 0.5625
Epoch 4/100
12003/12028 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - accuracy: 0.8247 -

In [ ]:
model.evaluate(test_keypoints, test_labels)

In [ ]:
import matplotlib.pyplot as plt

# Plot Utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

In [ ]:
model = tf.keras.models.load_model(checkpoint_path)

keypoints = np.array(annotations[0]['pose2'])

keypoints = [[0] * 2] * num_keypoints
keypoints += [keypoint[:2] for keypoint in keypoints]

keypoints = np.array(keypoints).astype(np.float32).reshape(num_keypoints * num_players * 2)

print(keypoints.shape)

max_x = max(keypoints)
normalized_keypoints = keypoints / max_x

prediction = model.predict(normalized_keypoints.reshape(1, num_keypoints * 2 * num_players))

print(prediction)

max_index = np.argmax(prediction[0])

for label, index in labels.items():
    if index == max_index:
        print(label)
        break

In [ ]:
# Calculate accuracy for each label

test_dict = {}

n = len(test_keypoints)

print(n)

for i in range(0, n):
    test = np.array([test_keypoints[i]])

    prediction = model.predict(test)
    
    correct_label = np.argmax(test_labels[i])
    predicted_label = np.argmax(prediction)
    
    label_name = list(labels.keys())[predicted_label]
    
    if predicted_label == correct_label:
        if label_name in test_dict:
            test_dict[label_name]["correct"] += 1
        else:
            test_dict[label_name] = {"correct": 1, "incorrect": 0}
    else:
        if label_name in test_dict:
            test_dict[label_name]["incorrect"] += 1
        else:
            test_dict[label_name] = {"correct": 0, "incorrect": 1}
    
for key in test_dict:
    correct = test_dict[key]["correct"]
    incorrect = test_dict[key]["incorrect"]
    
    print(f"Accuracy for {key}: {correct / (correct + incorrect)}")
